In [6]:
import os
import json
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI()

In [7]:
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [8]:
# create assistant
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4-1106-preview",
)
show_json(assistant)

{'id': 'asst_chItnl9hJfTGhdLqQko8Q8po',
 'created_at': 1699921492,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

In [ ]:
thread = client.beta.threads.create()
show_json(thread)